In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

secret_value_0 = user_secrets.get_secret(" HUGGINGFACE_TOKEN")
login(token = secret_value_0)

In [3]:
# Model
base_model_url = "/kaggle/input/llama-3.2/transformers/3b-instruct/1"
new_model_url = "/kaggle/input/fine-tune-llama-3-2-on-customer-support/llama-3.2-3b-it-Ecommerce-ChatBot/"

In [4]:
!pip install --upgrade transformers
!pip install --upgrade huggingface_hub


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format

2025-01-18 09:47:48.058683: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-18 09:47:48.058826: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-18 09:47:48.184736: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model_url)

base_model_reload= AutoModelForCausalLM.from_pretrained(
    base_model_url,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# Merge adapter with base model
tokenizer.chat_template = None
base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)
model = PeftModel.from_pretrained(base_model_reload, new_model_url)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [11]:
model = model.merge_and_unload()

AttributeError: 'LlamaForCausalLM' object has no attribute 'merge_and_unload'

In [12]:
instruction = """You are a top-rated customer service agent named John. 
    Be polite to customers and answer all their questions.
    """

messages = [{"role": "system", "content": instruction},
    {"role": "user", "content": "I have to see what payment payment modalities are accepted"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


I'm in tune with the fact that you would like to know the accepted payment methods. Let me assist you with that. We offer a variety of payment options to cater to different preferences and needs. Here are the payment methods we accept:

- Credit/Debit Card: We accept major credit and debit cards like Visa, Mastercard, and American Express.
- PayPal: A secure and widely recognized online payment platform.
- Bank Transfer: You can make direct bank transfers for a more secure and private payment experience.
- Apple Pay: For Apple device users, Apple Pay offers a convenient and fast payment method.
- Google Pay: Google's payment system allows for seamless transactions for Google users.

These are the primary payment methods we accept. If you have


In [13]:
new_model = "llama-3.2-3b-it-Ecommerce-ChatBot"

model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('llama-3.2-3b-it-Ecommerce-ChatBot/tokenizer_config.json',
 'llama-3.2-3b-it-Ecommerce-ChatBot/special_tokens_map.json',
 'llama-3.2-3b-it-Ecommerce-ChatBot/tokenizer.json')

In [14]:
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AbdulHadiDev/llama-3.2-3b-it-Ecommerce-ChatBot/commit/951a867449c1e91eb62549f29b620b1c66ad8c8e', commit_message='Upload tokenizer', commit_description='', oid='951a867449c1e91eb62549f29b620b1c66ad8c8e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AbdulHadiDev/llama-3.2-3b-it-Ecommerce-ChatBot', endpoint='https://huggingface.co', repo_type='model', repo_id='AbdulHadiDev/llama-3.2-3b-it-Ecommerce-ChatBot'), pr_revision=None, pr_num=None)